In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F


Generate random input data

In [52]:
y_fullyCon = torch.rand(5,129, 8)
y_conv = torch.rand(5, 1, 129,8, ) #5 input of dimension 128 by 8

print(y.shape)

print(y[1])


torch.Size([5, 129, 8])
tensor([[0.0694, 0.7527, 0.3935,  ..., 0.3756, 0.0466, 0.9681],
        [0.6232, 0.3888, 0.3341,  ..., 0.1659, 0.5252, 0.2928],
        [0.8684, 0.1892, 0.8716,  ..., 0.0564, 0.6973, 0.5204],
        ...,
        [0.9446, 0.1724, 0.6466,  ..., 0.3227, 0.8031, 0.9446],
        [0.2870, 0.3009, 0.9923,  ..., 0.2137, 0.2265, 0.9059],
        [0.6784, 0.6201, 0.4648,  ..., 0.3305, 0.9761, 0.8859]])


Define the architecture for the fully connected model

In [46]:
class FullyConnectedBaseline(nn.Module):
    def __init__(self, n_features, n_segments):
        super(FullyConnectedBaseline, self).__init__()
        self.fc1 = nn.Linear(n_features*n_segments, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc2_bn = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, n_features)

    def forward(self, x):
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.relu(self.fc2_bn(self.fc2(x)))
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Define the architecture for the convolutional model

In [73]:
class ConvolutionalBaseline(nn.Module):
    def __init__(self, n_features, n_segments):
        super(ConvolutionalBaseline, self).__init__()
        self.n_features = n_features
        self.n_segments = n_segments
        self.conv1 = nn.Conv2d(1, 18, kernel_size=(9,8), padding=(4,0))
        self.conv1_bn = nn.BatchNorm2d(18)
        self.conv2 = nn.Conv2d(18, 30, kernel_size=(9,1), padding=(4,0))
        self.conv2_bn = nn.BatchNorm2d(30)
        self.conv3 = nn.Conv2d(30, 40, kernel_size=(9,1), padding=(4,0))
        self.conv3_bn = nn.BatchNorm2d(40)
        self.fc1 = nn.Linear(40 * n_features, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, n_features)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3(x)))
        
        x = x.view(-1, 40 * self.n_features)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        return x

In [74]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


model = ConvolutionalBaseline(129, 8)

model.apply(init_weights)

loss_list = []

#x = y.view(5, -1)
#print(x)
#print(x.shape)
result = model.forward(y_conv) 

for param in model.parameters():
    print(param.data)

print(result.shape)
result.shape


tensor([[[[-8.9638e-02,  6.1142e-02,  1.0468e-01,  ..., -1.0599e-01,
           -8.5303e-02, -1.1750e-01],
          [ 9.3161e-02,  9.4978e-02, -1.8650e-02,  ..., -8.4085e-02,
            9.7468e-02, -4.4019e-02],
          [ 9.1588e-02,  1.6567e-02,  4.6962e-02,  ...,  8.7230e-02,
           -1.6789e-02, -3.2320e-02],
          ...,
          [ 7.3934e-02,  1.0283e-02, -3.4920e-02,  ..., -1.0553e-01,
            9.2206e-02, -2.5950e-02],
          [ 6.4461e-02, -6.4257e-02, -5.8587e-02,  ..., -8.7557e-03,
           -4.8704e-02, -6.3134e-02],
          [-4.2153e-02,  2.5621e-02,  7.9599e-02,  ...,  1.1381e-01,
           -2.8990e-02, -2.9843e-02]]],


        [[[ 4.5463e-02,  1.0437e-01,  5.9573e-02,  ..., -8.8465e-03,
            1.1404e-02, -1.7577e-02],
          [-3.5161e-02, -8.9446e-03, -9.0654e-02,  ..., -1.1756e-01,
            7.8539e-02, -4.1245e-02],
          [ 4.1944e-02,  8.3111e-02,  4.2807e-02,  ...,  7.7886e-03,
            8.1190e-02, -7.0768e-03],
          ...,
   

torch.Size([5, 129])